### Resources:
- [Queues and Input Pipelines](http://blog.altoros.com/pose-estimation-queues-and-input-pipelines-in-tensorflow.html)
- [Indico](https://indico.io/blog/tensorflow-data-inputs-part1-placeholders-protobufs-queues/)
- https://www.youtube.com/watch?v=wuo4JdG3SvU&index=1&list=PL9Hr9sNUjfsmEu1ZniY0XpHSzl5uihcXZ&spfreload=10

# Batch

### = tf.nn.batch_normalization()...
- [tf.nn.batch_normalization(x, mean, variance, offset, scale, variance_epsilon, name=None)](https://www.tensorflow.org/versions/r0.11/api_docs/python/nn.html#batch_normalization)

### = Custom Function...

In [ ]:
for i in range(1000):
  batch = mnist.train.next_batch(100)
  train_step.run(feed_dict={x: batch[0], y_: batch[1]})

In [ ]:
def optimize(num_iterations):
    for i in range(num_iterations):
        # Get a batch of training examples.
        # x_batch now holds a batch of images and
        # y_true_batch are the true labels for those images.
        x_batch, y_true_batch = data.train.next_batch(batch_size)
        
        # Put the batch into a dict with the proper names
        # for placeholder variables in the TensorFlow graph.
        # Note that the placeholder for y_true_cls is not set
        # because it is not used during training.
        feed_dict_train = {x: x_batch,
                           y_true: y_true_batch}

        # Run the optimizer using this batch of training data.
        # TensorFlow assigns the variables in feed_dict_train
        # to the placeholder variables and then runs the optimizer.
        session.run(optimizer, feed_dict=feed_dict_train)

# Readers

In [ ]:
Readers
https://www.tensorflow.org/versions/r0.11/how_tos/reading_data/index.html

import tensorflow as tf

There are three main methods of getting data into a TensorFlow program:
- Feeding: Python code provides the data when running each step.
- Reading from files: an input pipeline reads the data from files at the beginning of a TensorFlow graph.

# Queue

### Basic Example

In [ ]:
# FIFO stands for "first in, first out" queue
q = tf.FIFOQueue(capacity=20, dtypes=tf.int32)

In [ ]:
enqueue_px = tf.placeholder(tf.int32)
enqueue_op = q.enqueue(enqueue_px)

In [ ]:
with tf.Session() as sess:
    for i in range(10):
        sess.run(enqueue_op, feed_dict={enqueue_px:i})
        print("Queue size is now:"+str(sess.run(q.size())))
    
    # Remember to close the queue with a run command
    sess.run(q.close())

### Multithread Example
- So we have a single queue but we can call session run calls in parallel

In [2]:
q = tf.FIFOQueue(capacity=20, dtypes=tf.float32)

random_number = tf.random_uniform(shape=())
enqueue_op = q.enqueue(random_number)

In [3]:
with tf.Session() as sess:
    
    print(sess.run(q.size())) # prints 0

    def run():
        for i in range(5):
            sess.run(enqueue_op)
    
    # We create a (2) python thread, t-at will radd random numbers to the same queue in parallell
    threads = [threading.Thread(target=run, args=()) for i in range(2)]
    [t.start() for t in threads]
    print(sess.run(q.size())) # prints 4
    time.sleep(0.5)
    print(sess.run(q.size())) # prints 10

0


NameError: name 'threading' is not defined

# Feed
- TensorFlow's feed mechanism lets you inject data into any Tensor in a computation graph.

# Reading from files
A typical pipeline for reading records from files has the following stages:
- The list of filenames
- Optional filename shuffling
- Optional epoch limit
- Filename queue
- A Reader for the file format
- A decoder for a record read by the reader
- Optional preprocessing
- Example queue

# File Names
- For the list of filenames, use either a constant string Tensor

### tf.train.match_filenames_once()...
- [tf.train.match_filenames_once(pattern, name=None)](https://www.tensorflow.org/versions/r0.11/api_docs/python/io_ops.html#match_filenames_once)

In [ ]:
# One option is a constant string tensor
tf.constant(["file0", "file1"], dtype=tf.string, shape=None, name='Const')

In [ ]:
# Save the list of files matching pattern, so it is only computed once.
tf.train.match_filenames_once(pattern, name=None)

# creates a FIFO queue
- A FIFO queue for holding the filenames until the reader needs them.
- FIFO is an acronym for first in, first out

### tf.train.string_input_producer()
- [tf.train.string_input_producer(string_tensor, num_epochs=None, shuffle=True, seed=None, capacity=32, shared_name=None, name=None)](https://www.tensorflow.org/versions/r0.11/api_docs/python/io_ops.html#string_input_producer)
- string_input_producer has options for **shuffling** and **setting a maximum number of epochs**

# FIFO is an acronym for first in, first out
tf.train.string_input_producer(string_tensor, num_epochs=None, shuffle=True, seed=None, capacity=32, shared_name=None, name=None)

In [ ]:
# We first create a "producer" queue.  It creates a production line which
# will queue up the file names and allow another queue to deque the file
# names all using a tf queue runner.
# Put simply, this is the entry point of the computational graph.
# It will generate the list of file names.
# We also specify it's capacity beforehand.
producer = tf.train.string_input_producer(
    files, capacity=len(files))

producer

In [ ]:
# We need something which can open the files and read its contents.
reader = tf.WholeFileReader()

In [ ]:
# We pass the filenames to this object which can read the file's contents.
# This will create another queue running which dequeues the previous queue.
keys, vals = reader.read(producer)

- [tf.train.string_input_producer(string_tensor, num_epochs=None, shuffle=True, seed=None, capacity=32, shared_name=None, name=None)](https://www.tensorflow.org/versions/r0.11/api_docs/python/io_ops.html#string_input_producer)

In [ ]:
with tf.Session() as sess:
  
  # initialize the variables
  sess.run(tf.initialize_all_variables())
  
  # initialize the queue threads to start to shovel data
  coord = tf.train.Coordinator()
  threads = tf.train.start_queue_runners(coord=coord)

  print "from the train set:"
  for i in range(20):
    print sess.run(train_label_batch)

  print "from the test set:"
  for i in range(10):
    print sess.run(test_label_batch)

  # stop our queue threads and properly close the session
  coord.request_stop()
  coord.join(threads)
  sess.close()

# TFRecords
- https://indico.io/blog/tensorflow-data-inputs-part1-placeholders-protobufs-queues/
- TFRecords are TensorFlow’s default data format. A record is simply a binary file that contains serialized tf.train.Example Protobuf objects, and can be created from Python in a few lines of code. Below is an example to convert mnist to this format.

In [4]:
# load up some dataset. Could be anything but skdata is convenient.
from skdata.mnist.views import OfficialVectorClassification
from tqdm import tqdm
import numpy as np
import tensorflow as tf

data = OfficialVectorClassification()
trIdx = data.sel_idxs[:]

# one MUST randomly shuffle data before putting it into one of these
# formats. Without this, one cannot make use of tensorflow's great
# out of core shuffling.
np.random.shuffle(trIdx)


writer = tf.python_io.TFRecordWriter("mnist.tfrecords")
# iterate over each example
# wrap with tqdm for a progress bar
for example_idx in tqdm(trIdx):
    features = data.all_vectors[example_idx]
    label = data.all_labels[example_idx]

    # construct the Example proto boject
    example = tf.train.Example(
        # Example contains a Features proto object
        features=tf.train.Features(
          # Features contains a map of string to Feature proto objects
          feature={
            # A Feature contains one of either a int64_list,
            # float_list, or bytes_list
            'label': tf.train.Feature(
                int64_list=tf.train.Int64List(value=[label])),
            'image': tf.train.Feature(
                int64_list=tf.train.Int64List(value=features.astype("int64"))),
    }))
    # use the proto object to serialize the example to a string
    serialized = example.SerializeToString()
    # write the serialized object to disk
    writer.write(serialized)

ImportError: No module named 'skdata'

### Embeddings
- Word embedding is the collective name for a set of language modeling and feature learning techniques in natural language processing (NLP) where words or phrases from the vocabulary are mapped to vectors of real numbers in a low-dimensional space relative to the vocabulary size ("continuous space").

In [ ]:
# we first create a variable to take us from our one-hot representation to our LSTM cells
embedding = tf.get_variable("embedding", [n_chars, n_cells])

# And then use tensorflow's embedding lookup to look up the ids in X
Xs = tf.nn.embedding_lookup(embedding, X)

# The resulting lookups are concatenated into a dense tensor
print(Xs.get_shape().as_list())

Great so we now have about 164 thousand characters and 85 unique characters in our vocabulary which we can use to help us train a model of language.  Rather than use the characters, we'll convert each character to a unique integer.  We'll later see that when we work with words, we can achieve a similar goal using a very popular model called word2vec: https://www.tensorflow.org/versions/r0.9/tutorials/word2vec/index.html

We'll first create a look up table which will map a character to an integer:

In [1]:
encoder = dict(zip(vocab, range(len(vocab))))
decoder = dict(zip(range(len(vocab)), vocab))

NameError: name 'vocab' is not defined

# Text

### = Word2Vec...